# Paso 1: Importar Librerías de Python


In [1]:
pip install scikit-surprise

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error

from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from keras.optimizers import Adam

# Paso 2: Lectura del Dataset

In [ ]:
!gdown '1ChzYRhSgS7ufpUR5qTGF1jIoNt5lM5Rx'

In [4]:
# Suponiendo que tu DataFrame se llama df
df = pd.read_csv('DataFinal_Amazon.csv') 

<ipython-input-4-1f8e3361d82a>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV


# Paso 3: Reducción de Datos


In [5]:
# Tomar una muestra del 30% del dataset, si es necesario
df = df.sample(frac=0.3, random_state=42)

# Paso 4: Ingeniería de Características (Feature Engineering)

In [6]:
# Convertir 'reviewDate' a formato datetime
df['reviewDate'] = pd.to_datetime(df['reviewDate'])

# Crear nuevas variables temporales
df['year'] = df['reviewDate'].dt.year
df['month'] = df['reviewDate'].dt.month
df['day'] = df['reviewDate'].dt.day
df['weekday'] = df['reviewDate'].dt.weekday

print(df[['reviewDate', 'year', 'month', 'day', 'weekday']].head())

        reviewDate  year  month  day  weekday
168877  2017-10-23  2017     10   23        0
4652188 2018-07-09  2018      7    9        0
799784  2016-03-03  2016      3    3        3
1025869 2017-01-10  2017      1   10        1
2337820 2015-06-08  2015      6    8        0


# Paso 5: Creación de Características


In [7]:
df['reviewText_length'] = df['reviewText'].apply(lambda x: len(str(x)))

# Paso 6: Limpieza/Transformación de Datos


In [8]:
# Eliminemos posibles duplicados:
df = df.drop_duplicates()

# Paso 10: Transformación de Datos

In [9]:
# Normalizar la columna de precios si es necesario:
df['price_normalized'] = (df['price'] - df['price'].mean()) / df['price'].std()

<ipython-input-9-c7b197adf89a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_normalized'] = (df['price'] - df['price'].mean()) / df['price'].std()


# Modelo de Filtrado Colaborativo usando Embedding con Keras:

In [10]:
#df.head(3)

In [ ]:
data = df

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Crear un LabelEncoder para cada columna
reviewerID_encoder = LabelEncoder()
asin_encoder = LabelEncoder()

# Ajustar y transformar las columnas
data['reviewerID_encoded'] = reviewerID_encoder.fit_transform(data['reviewerID'])
data['asin_encoded'] = asin_encoder.fit_transform(data['asin'])

# Mostrar las primeras filas del DataFrame para verificar
print(data[['reviewerID', 'reviewerID_encoded', 'asin', 'asin_encoded']].head())

       reviewerID  reviewerID_encoded        asin  asin_encoded
0  A1BB77SEBQT8VX              121313  B00007GDFV            39
1  A1BB77SEBQT8VX              121313  B00007GDFV            39
2   AHWOW7D1ABO9C             1265535  B00007GDFV            39
3   AHWOW7D1ABO9C             1265535  B00007GDFV            39
4   AKS3GULZE0HFC             1296109  B00007GDFV            39


In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Embedding, Input, Dot, Flatten, Dense

# Número de usuarios únicos y número de ítems únicos
n_users = len(np.unique(data['reviewerID_encoded']))
n_items = len(np.unique(data['asin_encoded']))

# Dimensiones del embedding
embedding_dim = 10

# Entradas
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

# Embeddings
user_embedding = Embedding(n_users, embedding_dim)(user_input)
item_embedding = Embedding(n_items, embedding_dim)(item_input)

# Producto punto para predecir la valoración/rating
merged = Dot(axes=2)([user_embedding, item_embedding])
merged = Flatten()(merged)

# Modelo
model = Model(inputs=[user_input, item_input], outputs=merged)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

verbose=0: No muestra ninguna barra de progreso ni métricas.

verbose=1: Muestra una barra de progreso y actualiza las métricas después de cada lote.

verbose=2: Muestra las métricas después de cada época, pero no muestra la barra de progreso.

In [ ]:
# Entrenamiento
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values
model.fit([user_ids, item_ids], ratings, epochs=3, batch_size=4096, verbose=1)

Epoch 1/3
1173/1173 [==============================] - 199s 169ms/step - loss: 19.4372 - mae: 4.2510
Epoch 2/3
1173/1173 [==============================] - 198s 169ms/step - loss: 18.6747 - mae: 4.1410
Epoch 3/3
1173/1173 [==============================] - 200s 171ms/step - loss: 17.1693 - mae: 3.9141


In [ ]:
predictions = model.predict([user_ids, item_ids])

150018/150018 [==============================] - 154s 1ms/step


In [ ]:
# MSE
mse = mean_squared_error(ratings, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(ratings, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((ratings - predictions.flatten()) / ratings)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 15.9169
RMSE: 3.9896
MAE: 3.7271
MAPE: 86.76%


In [ ]:
# Entrenamiento
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values
model.fit([user_ids, item_ids], ratings, epochs=1, batch_size=2048, verbose=1)

2345/2345 [==============================] - 391s 166ms/step - loss: 14.1258 - mae: 3.4392


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

predictions = model.predict([user_ids, item_ids])

150018/150018 [==============================] - 154s 1ms/step


In [ ]:
# MSE
mse = mean_squared_error(ratings, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(ratings, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((ratings - predictions.flatten()) / ratings)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 12.6531
RMSE: 3.5571
MAE: 3.1998
MAPE: 73.87%


In [ ]:
from sklearn.model_selection import train_test_split

# Crear los arrays con los datos
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values

# Dividir en train y test
(user_ids_train, user_ids_test,
 item_ids_train, item_ids_test,
 ratings_train, ratings_test) = train_test_split(user_ids, item_ids, ratings, test_size=0.3, random_state=42)

In [ ]:
# Entrenar el modelo con el conjunto de entrenamiento
model.fit([user_ids_train, item_ids_train], ratings_train, epochs=5, batch_size=2048, verbose=1)

Epoch 1/5
1641/1641 [==============================] - 285s 174ms/step - loss: 11.1945 - mae: 2.9552
Epoch 2/5
1641/1641 [==============================] - 282s 172ms/step - loss: 9.0798 - mae: 2.5762
Epoch 3/5
1641/1641 [==============================] - 280s 171ms/step - loss: 7.1838 - mae: 2.2069
Epoch 4/5
1641/1641 [==============================] - 279s 170ms/step - loss: 5.6018 - mae: 1.8735
Epoch 5/5
1641/1641 [==============================] - 280s 171ms/step - loss: 4.3327 - mae: 1.5843


In [ ]:
# Evaluar el modelo con el conjunto de prueba (opcional)
loss, mae = model.evaluate([user_ids_test, item_ids_test], ratings_test, batch_size=2048)
print(f'Test Loss: {loss:.4f}')
print(f'Test MAE: {mae:.4f}')

704/704 [==============================] - 1s 2ms/step - loss: 7.0447 - mae: 2.1847
Test Loss: 7.0447
Test MAE: 2.1847


In [ ]:
predictions = model.predict([user_ids_test, item_ids_test])

45006/45006 [==============================] - 48s 1ms/step


In [ ]:
# MSE
mse = mean_squared_error(ratings_test, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(ratings_test, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((ratings_test - predictions.flatten()) / ratings_test)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 7.0447
RMSE: 2.6542
MAE: 2.1847
MAPE: 52.60%


# Modelo de Filtrado Colaborativo usando SVD

Utilizaremos SVD para hacer recomendaciones basadas en interacciones pasadas.


In [11]:
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

# Leer y procesar datos
reader = Reader()
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

# Dividir el dataset
trainset, testset = train_test_split(data, test_size=0.3, random_state=10)

# Entrenar el modelo SVD
svd = SVD()
svd.fit(trainset)
predictions_svd = svd.test(testset)

# Función para calcular el Mean Absolute Percentage Error (MAPE)
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Evaluar el modelo
rmse_svd = accuracy.rmse(predictions_svd, verbose=True)
mse_svd = mean_squared_error([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])
mae_svd = mean_absolute_error([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])
mape_svd = mape([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])

print(f"SVD - RMSE: {rmse_svd}, MSE: {mse_svd}, MAE: {mae_svd}, MAPE: {mape_svd}")

RMSE: 1.1267
SVD - RMSE: 1.1266590735004887, MSE: 1.2693606679009797, MAE: 0.8692695424111369, MAPE: 36.112821495553185


# Modelo de Filtrado Colaborativo usando Embedding y Red Neuronal Multicapa con Keras:

Otra enfoque popular para filtrado colaborativo es utilizar redes neuronales multicapa, que esencialmente aprenden características no lineales de los datos. Vamos a construir un modelo que fusiona los embeddings de usuarios y artículos (por ejemplo, películas) en una red neuronal densa.

Este enfoque, que combina embeddings y redes neuronales multicapa, puede capturar interacciones más complejas y no lineales entre usuarios y artículos. Sin embargo, es crucial prestar atención al sobreajuste y asegurarse de que el modelo no esté simplemente memorizando los datos. Por lo tanto, es recomendable emplear técnicas de regularización, ajustar hiperparámetros y validar el rendimiento con un conjunto de datos de validación.

In [11]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from keras.optimizers import Adam
import pandas as pd

# Convertir reviewerID y asin a índices numéricos secuenciales
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
n_latent_factors = 50  # Número de factores latentes
hidden_units = [128, 64]  # Unidades en las capas ocultas
dropout_rate = 0.2

# Arquitectura del modelo
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')
user_embedding = Embedding(n_users, n_latent_factors, name='user_embedding')(user_input)
item_embedding = Embedding(n_items, n_latent_factors, name='item_embedding')(item_input)
user_vec = Flatten(name='flatten_users')(user_embedding)
item_vec = Flatten(name='flatten_items')(item_embedding)
concat = Concatenate()([user_vec, item_vec])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu')(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1)(dense)
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [12]:
# Entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# Usando validation_split, divide los datos
train_size = int(0.8 * len(user_data))
user_data_train, user_data_val = user_data[:train_size], user_data[train_size:]
item_data_train, item_data_val = item_data[:train_size], item_data[train_size:]
y_true = rating_data[train_size:]

history = model.fit([user_data_train, item_data_train], rating_data[:train_size],
                    epochs=5, validation_data=([user_data_val, item_data_val], y_true),
                    batch_size=2048, verbose=1)

# Predicciones
predictions = model.predict([user_data_val, item_data_val])

Epoch 1/5
557/557 [==============================] - 27s 38ms/step - loss: 2.8788 - val_loss: 1.3066
Epoch 2/5
557/557 [==============================] - 9s 16ms/step - loss: 1.2273 - val_loss: 1.3545
Epoch 3/5
557/557 [==============================] - 9s 15ms/step - loss: 0.8625 - val_loss: 1.4520
Epoch 4/5
557/557 [==============================] - 9s 16ms/step - loss: 0.6888 - val_loss: 1.4685
Epoch 5/5
8909/8909 [==============================] - 12s 1ms/step


In [17]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 1.508964377056769
RMSE: 1.2283991114685686
MAE: 0.9115688753774837
MAPE: 40.61646684475661%


Estas métricas te ayudarán a tener una mejor idea del desempeño de tu modelo. Por ejemplo:

MSE y RMSE son útiles cuando quieres penalizar grandes errores.

MAE te da una idea del error medio sin considerar la dirección del error.

MAPE es útil cuando quieres representar el error en términos porcentuales.

Para una evaluación completa, es recomendable utilizar un conjunto de validación aparte (es decir, no solo depender del validation_split). Esto asegura que estás evaluando el desempeño en datos que el modelo nunca ha visto durante el entrenamiento.